In [2]:
import numpy as np
from itertools import combinations
from collections import Counter

#### Descrição do problema

Dada uma rodada $R$ do Poker Texas Hold'em, sua mão $M$ é composta por duas cartas $C_1$ e $C_2$. Qual a probabilidade aproximada de você perder?

### Criando as Classes Carta e Jogador

In [3]:
class Carta:
    VALORES = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
    NAIPES = ["Copas", "Ouros", "Espadas", "Paus"]

    def __init__(self, valor, naipe):
        if valor in Carta.VALORES:
            self.valor = valor
        else:
            raise ValueError(f"Valor inválido: {valor}")
        
        if naipe in Carta.NAIPES:
            self.naipe = naipe
        else:
            raise ValueError(f"Naipe inválido: {naipe}")
    
    def __str__(self):
        return f"{self.valor} de {self.naipe}"
    
class Jogador:
    def __init__(self, nome, carta1, carta2):
        self.nome = nome
        self.carta1 = carta1
        self.carta2 = carta2

    def __str__(self):
        return f"Jogador {self.nome} com cartas: {self.carta1} e {self.carta2}"


### Variáveis, listas e dicionários auxiliares

In [4]:
valor_map = {
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6": 6,
        "7": 7,
        "8": 8,
        "9": 9,
        "10": 10,
        "J": 11,
        "Q": 12,
        "K": 13,
        "A": 14  # Considerando o Ás como o maior valor
        }

hand_map = [
    'royal_flush',      # sequencia maior de mesmo naipe
    'straight_flush',   # sequencia do mesmo naipe
    'foak',             # quadra
    'full_house',       # trinca e dupla
    'flush',            # mesmo naipe
    'straight',         # sequencia
    'toak',             # trinca 
    'two_pair',         # dois pares
    'pair',             # um par
    'highest'           # maior carta
]

valores_array = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
naipes_array = ["Copas", "Ouros", "Espadas", "Paus"]
naipe_map = {"Copas":0, "Ouros":1, "Espadas":2, "Paus":3}
valores_map = {"A":0, "2":1, "3":2, "4":3, "5":4, "6":5, "7":6, "8":7, "9":8, "10":9, "J":10, "Q":11, "K":12}

### Funções de Combinações de 5 cartas
Dada 5 cartas, a função $f$ recebe as cartas e devolve, caso as cinco cartas formem a combinação, um valor entre $2$ e $14$ que corresponde ao valor da maior carta, ou devolve 0 caso contrário. O caso do Royal Flush é uma execeção, uma vez que é a melhor combinação, e, por isso, devolve $True$ ou $False$

In [5]:

def royal_flush(carta1,carta2,carta3,carta4,carta5):
    royal = {'A', 'J', 'Q', 'K', '10'}
    if (carta1.naipe == carta2.naipe == carta3.naipe == carta4.naipe == carta5.naipe):
        valores_cartas = {carta1.valor, carta2.valor, carta3.valor, carta4.valor, carta5.valor}
        return valores_cartas == royal
    return False

def straight_flush(carta1,carta2,carta3,carta4,carta5):
    if (carta1.naipe == carta2.naipe == carta3.naipe == carta4.naipe == carta5.naipe):
        cartas = [carta1,carta2,carta3,carta4,carta5]
        valores = [valor_map[carta.valor] for carta in cartas]
        valores.sort()
        for i in range(4):
            if valores[i] + 1 != valores[i + 1]:
                return 0
        return valores[-1]
    return 0

def foak(carta1,carta2,carta3,carta4,carta5):
    four = Counter([carta1.valor, carta2.valor, carta3.valor, carta4.valor, carta5.valor])
    return 4 in four.values()

def full_house(carta1,carta2,carta3,carta4,carta5):
    counter = Counter([carta1.valor, carta2.valor, carta3.valor, carta4.valor, carta5.valor])
    count_freq = Counter(counter.values())
    if count_freq.get(3, 0) == 1 and count_freq.get(2, 0) == 1:
        three = max(counter, key=counter.get)
        return valor_map[three]
    return 0

def flush(carta1,carta2,carta3,carta4,carta5):
    if (carta1.naipe == carta2.naipe == carta3.naipe == carta4.naipe == carta5.naipe):
        cartas = [carta1,carta2,carta3,carta4,carta5]
        valores = [valor_map[carta.valor] for carta in cartas]
        valores.sort()
        return valores[-1]
    return 0

def straight(carta1,carta2,carta3,carta4,carta5):
    if (carta1.naipe == carta2.naipe == carta3.naipe == carta4.naipe == carta5.naipe):
        return 0
    cartas = [carta1,carta2,carta3,carta4,carta5]
    valores = [valor_map[carta.valor] for carta in cartas]
    valores.sort()
    for i in range(4):
        if valores[i] + 1 != valores[i + 1]:
            return 0
    return valores[-1]

def toak(carta1,carta2,carta3,carta4,carta5):
    four = Counter([carta1.valor, carta2.valor, carta3.valor, carta4.valor, carta5.valor])
    return 3 in four.values()

def two_pair(carta1,carta2,carta3,carta4,carta5):
    counter = Counter([carta1.valor, carta2.valor, carta3.valor, carta4.valor, carta5.valor])
    count_freq = Counter(counter.values())
    if count_freq.get(2, 0) == 2:
        for key in list(counter.keys()):
            if counter[key] == 1:
                del counter[key]
        valores = [valor_map[carta] for carta in list(counter.keys())]
        valores.sort()
        return valores[-1]
    return 0
def pair(carta1,carta2,carta3,carta4,carta5):
    counter = Counter([carta1.valor, carta2.valor, carta3.valor, carta4.valor, carta5.valor])
    count_freq = Counter(counter.values())
    if count_freq.get(2,0) == 1 and count_freq.get(3,0) != 1:
        pair = max(counter, key=counter.get)
        return valor_map[pair]
    return 0

def highest(carta1,carta2,carta3,carta4,carta5):
    cartas = [carta1,carta2,carta3,carta4,carta5]
    valores = [valor_map[carta.valor] for carta in cartas]
    valores.sort()
    return valores[-1]

### Best Hand
Dada uma certa rodada, as cartas compartilhadas podem ter diversos tamanhos: podem ter nenhuma carta na mesa (Pré-Flop), 3 cartas (Flop), 4 cartas (Turn) ou 5 cartas (River). <br>
Nas últimas três opções, a função `best_hand` recebe as duas cartas da sua mão, na variável `mao` e as cartas compartilhadas, independentemente se está no flop, turn, ou river, e devolve um número `best_index` e `best_point`, que podem ser usadas para saber qual é a melhor combinação que as cartas compoem, juntamente com a maior carta

In [6]:
def best_hand(mao, river):
    conj = mao + river
    subsets = list(combinations(conj, 5))
    best_index = 10
    best_point = 0
    for five in subsets:
        possible_hands = [royal_flush(five[0],five[1],five[2],five[3],five[4]),
                        straight_flush(five[0],five[1],five[2],five[3],five[4]),
                        foak(five[0],five[1],five[2],five[3],five[4]),
                        full_house(five[0],five[1],five[2],five[3],five[4]),
                        flush(five[0],five[1],five[2],five[3],five[4]),
                        straight(five[0],five[1],five[2],five[3],five[4]),
                        toak(five[0],five[1],five[2],five[3],five[4]),
                        two_pair(five[0],five[1],five[2],five[3],five[4]),
                        pair(five[0],five[1],five[2],five[3],five[4]),
                        highest(five[0],five[1],five[2],five[3],five[4])]
        for index, hand in enumerate(possible_hands):
            if hand and index <= best_index:
                
                best_point = hand
                best_index = index
                continue
    return best_index, best_point


### Teste 1:
Recebi um par de Ás e o flop é 4 de Ouros, 2 de Copas, e 6 de Copas.
A função `best_hand` devolveu `'pair'`, com pontuação `14`, isto é, um par de Ás.

In [7]:
carta1 = Carta("A", "Copas")
carta2 = Carta("A", "Ouros")
carta3 = Carta("4", "Ouros")
carta4 = Carta("2", "Copas")
carta5 = Carta("6", "Copas")
#carta6 = Carta("4", "Ouros")
#carta7 = Carta("2", "Espadas")

mao = [carta1, carta2]
turn = [carta3, carta4, carta5]
best_index, point = best_hand(mao, turn)
print(hand_map[best_index], point)

pair 14


Para as próximas funções, será necessário montar o baralho completo, para conseguir os possíveis pares dos inimigos:

In [8]:
deck = [] # matriz de carta naipe x valor

valores_array = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
naipes_array = ["Copas", "Ouros", "Espadas", "Paus"]
naipe_map = {"Copas":0, "Ouros":1, "Espadas":2, "Paus":3}
valores_map = {"A":0, "2":1, "3":2, "4":3, "5":4, "6":5, "7":6, "8":7, "9":8, "10":9, "J":10, "Q":11, "K":12}

for naipe in naipes_array:
    n = []
    for val in valores_array:
        n.append(Carta(val, naipe))
    deck.append(n)




### a função all_pairs_possible
Esta função pega um baralho modificado, na variável `deck_op`, e monta todos os possíveis pares entre as cartas

In [9]:
def all_pairs_possible(deck_op):
    all_elements = [element for row in deck_op for element in row if element != 0]

    # Generate all possible pairs using combinations
    pairs = list(combinations(all_elements, 2))
    
    return pairs

### Probabilidade de algum jogador vencer:
Esta função recebe sua mão `your_hand`, sua melhor combinação `best_hand_index` e `best_point` e o `flop`.
Devolve a probabilidade: 
$$P_{perder}=\frac{\text{Pares Vencedores}}{\text{Todos os pares possíveis}}=\sum{\frac{\text{Par Vencedor}}{\binom{52-2-3}{2}}}$$

In [10]:
def prob_of_opponent_winning_flop(your_hand, best_hand_index,best_point, flop): # len(flop) = 3
    #print('Sua mão é um',your_hand[0].valor,'de',your_hand[0].naipe,'e um',your_hand[1].valor,'de',your_hand[1].naipe)
    #print('Sua melhor mão é um', hand_map[best_index], 'com pontuação', best_point, 'com index', best_hand_index)
    round = len(flop)
    winning_count = 0

    deck_op = deck

    for card in your_hand:
        naipe = card.naipe
        val = card.valor
        print()
        deck_op[naipe_map[naipe]][valores_map[val]] = 0

    for card in flop:
        naipe = card.naipe
        val = card.valor

        deck_op[naipe_map[naipe]][valores_map[val]] = 0

    pairs = all_pairs_possible(deck_op)
    print(np.array(pairs).shape)
    for par in pairs:
        #print(par[0].valor,'de', par[0].naipe, 'junto com',par[1].valor,'de', par[1].naipe)
        five= flop + list(par)
        
        #subsets = list(combinations(conj, 5))
        possible_hands = [royal_flush(five[0],five[1],five[2],five[3],five[4]),
                            straight_flush(five[0],five[1],five[2],five[3],five[4]),
                            foak(five[0],five[1],five[2],five[3],five[4]),
                            full_house(five[0],five[1],five[2],five[3],five[4]),
                            flush(five[0],five[1],five[2],five[3],five[4]),
                            straight(five[0],five[1],five[2],five[3],five[4]),
                            toak(five[0],five[1],five[2],five[3],five[4]),
                            two_pair(five[0],five[1],five[2],five[3],five[4]),
                            pair(five[0],five[1],five[2],five[3],five[4]),
                            highest(five[0],five[1],five[2],five[3],five[4])]
        for i in range(best_hand_index+1):
            if (possible_hands[i] > best_point and i == best_hand_index) or (i < best_hand_index and possible_hands[i] > 0):
                winning_count += 1
                break
            
    all = int((52 - 2 - round) * (52 - 3 - round)/2)
    return winning_count/all

        
                    

In [11]:
p = prob_of_opponent_winning_flop(mao, best_index, point, turn)
print(p)



(1081, 2)
0.04810360777058279
